In [497]:
import pandas as pd

In [498]:
#read in the json file

data_file = "purchase_data.json"
purchase_data = pd.read_json(data_file)

In [499]:
#add an age category column for later
bins = [0, 9, 14, 19, 24, 29, 34, 39, 1000]
group_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34','35-39','40+']
purchase_data["Age Group"]=pd.cut(purchase_data["Age"], bins, labels=group_names)

#create a dataframe with only unique players and count them
unique_players = purchase_data.drop_duplicates(subset=['SN'], keep='first')
players_count = unique_players.count()[5]

#turn count of unique players into a dataframe
total_players = pd.DataFrame({"Total Players":players_count}, index=[0])
total_players

,Total Players
0,573


In [500]:
#calculate number of unique items
unique_items = purchase_data.drop_duplicates(subset=['Item ID'], keep='first').count()[2]
unique_items

183

In [556]:
#calculate the average price of items
average_purchase_price = "${0:.2f}".format(purchase_data['Price'].mean())

#calculate total purchases
total_purchases = purchase_data.count()[1]

#calculate total revenue
total_revenue = "${0:.2f}".format(purchase_data['Price'].sum())

In [564]:
#create summary dataframe
summary_table = pd.DataFrame({"Number of Unique Items":unique_items, "Average Price":average_purchase_price, "Number of Purchases":total_purchases, "Total Revenue": total_revenue}, index=[0])
summary_table = summary_table[["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"]]
summary_table

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$2.93,780,$2286.33


In [586]:
#Find the percent and number of unique players by gender
genders = unique_players["Gender"].value_counts()
male_players = genders[0]
percent_male = "{0:.2f}%".format((male_players/players_count)*100)
female_players = genders[1]
percent_female = "{0:.2f}%".format((female_players/players_count)*100)
other_players = genders[2]
percent_other = "{0:.2f}%".format((other_players/players_count)*100)

In [587]:
genders_count = pd.DataFrame({"Gender":["Male", "Female", "Other/Non-Disclosed"], "Percentage of Players":[percent_male, percent_female, percent_other], "Total Count":[male_players, female_players, other_players]})
genders_count.set_index("Gender")


,Percentage of Players,Total Count
Gender,,
Male,81.15%,465
Female,17.45%,100
Other/Non-Disclosed,1.40%,8


In [505]:
gg = purchase_data.groupby(["Gender"])

In [506]:
purchases_by_g = gg['Age'].count()
price_by_g = round(gg['Price'].mean(),2).map("${:,.2f}".format)
revenue_by_g = gg['Price'].sum().map("${:,.2f}".format)
normalized_by_g = round(gg['Price'].sum()/genders,2).map("${:,.2f}".format)

In [592]:
genders_summary = pd.DataFrame({"Purchase Count": purchases_by_g, "Average Purchase Price": price_by_g, "Total Purchase Value":revenue_by_g, "Normalized Totals": normalized_by_g})
genders_summary = genders_summary[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
genders_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$3.83
Male,633,$2.95,"$1,867.68",$4.02
Other / Non-Disclosed,11,$3.25,$35.74,$4.47


In [593]:
#Find the percent of total players and player count for each age group
by_age = purchase_data.groupby(['Age Group'])
unique_by_age = unique_players.groupby(['Age Group'])
players_by_a = unique_by_age['SN'].count()
age_group_count = pd.DataFrame({"Total Count":players_by_a, "Percent of Players":((players_by_a/players_count)*100).map("{0:.2f}%".format)})
age_group_count

,Percent of Players,Total Count
Age Group,,
<10,3.32%,19
10-14,4.01%,23
15-19,17.45%,100
20-24,45.20%,259
25-29,15.18%,87
30-34,8.20%,47
35-39,4.71%,27
40+,1.92%,11


In [594]:
players_in_age_group = unique_players["Age Group"].value_counts()
purchases_by_a = by_age['Age'].count()
price_by_a = by_age['Price'].mean().map("${:,.2f}".format)
revenue_by_a = by_age['Price'].sum().map("${:,.2f}".format)
normalized_by_a = (by_age['Price'].sum()/players_in_age_group).map("${:,.2f}".format)

age_summary = pd.DataFrame({"Purchase Count": purchases_by_a, "Average Purchase Price": price_by_a, "Total Purchase Value":revenue_by_a, "Normalized Totals": normalized_by_a})
age_summary = age_summary[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
age_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
10-14,35,$2.77,$96.95,$4.22
15-19,133,$2.91,$386.42,$3.86
20-24,336,$2.91,$978.77,$3.78
25-29,125,$2.96,$370.33,$4.26
30-34,64,$3.08,$197.25,$4.20
35-39,42,$2.84,$119.40,$4.42
40+,17,$3.16,$53.75,$4.89
<10,28,$2.98,$83.46,$4.39


In [595]:
by_sn = purchase_data.groupby(['SN'])
purchases_by_sn = by_sn['Age'].count()
price_by_sn = by_sn['Price'].mean().map("${:,.2f}".format)
revenue_by_sn = by_sn['Price'].sum()

##create data frame of top 5 items sorted by Total Purchase Value
top_spenders = pd.DataFrame({"Purchase Count": purchases_by_sn, "Average Purchase Price": price_by_sn, "Total Purchase Value":revenue_by_sn})
top_spenders_sorted = top_spenders.sort_values('Total Purchase Value', ascending=False).head(5)

#Total Purchase Value must be formated after sorting
top_spenders_sorted["Total Purchase Value"] = top_spenders_sorted["Total Purchase Value"].map("${:,.2f}".format)
top_spenders_sorted = top_spenders_sorted[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]
top_spenders_sorted

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


In [596]:
by_item = purchase_data.groupby(['Item ID', 'Item Name'])
purchases_by_item = by_item['Age'].count()
price_by_item = by_item['Price'].mean().map("${:,.2f}".format)
revenue_by_item = by_item['Price'].sum()

#Create top items dataframe
top_items = pd.DataFrame({"Purchase Count": purchases_by_item, "Item Price": price_by_item, "Total Purchase Value":revenue_by_item})

#Create dataframe of top 5 items sorted by purchase count
by_count = top_items.sort_values('Purchase Count', ascending=False).head(5)

#Format the Total Purchase Value column
by_count["Total Purchase Value"] = by_count["Total Purchase Value"].map("${:,.2f}".format)
by_count = by_count[["Purchase Count", 'Item Price', "Total Purchase Value"]]
by_count

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


In [597]:
#create data frame of top 5 items sorted by Total Purchase Value
by_total_value = top_items.sort_values('Total Purchase Value', ascending=False).head(5)

#Total Purchase Value must be formated after sorting
by_total_value["Total Purchase Value"] = by_total_value["Total Purchase Value"].map("${:,.2f}".format)
by_total_value = by_total_value[["Purchase Count", 'Item Price', "Total Purchase Value"]]
by_total_value

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
